In [15]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import matplotlib as plt
import starter
import importlib
importlib.reload(starter)

<module 'starter' from 'c:\\Users\\erica\\OneDrive - ETH Zurich\\Exchange\\Machine Learning\\CS349---Group8\\HW2\\starter.py'>

## Part I

In [4]:
# dimensionality reduction
def reduce(train, valid, test, method):
    if method == 'pca':
        return starter.pca(train, valid, test)
    elif method == 'bin':
        return starter.binarize(train, valid, test)
    elif method == 'none':
        return starter.make_float(train, valid, test)
    
# test model
def evaluate_knn(train, query, distance_metric):

    labels=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    query_labels = [i[0] for i in query]
    query_data = [i[1] for i in query]
    result_labels = starter.knn(train, query_data, distance_metric)

    # performance metrics
    query_accuracy = accuracy_score(query_labels, result_labels)
    query_precision = precision_score(query_labels, result_labels, average='macro')
    query_recall = recall_score(query_labels, result_labels, average='macro')
    query_f1 = f1_score(query_labels, result_labels, average='macro')
    confusion = confusion_matrix(query_labels, result_labels, labels=labels)

    return query_accuracy, query_precision, query_recall, query_f1, confusion

In [5]:
mnist_train = starter.read_data('mnist_train.csv')
mnist_valid = starter.read_data('mnist_valid.csv')
mnist_test = starter.read_data('mnist_test.csv')

In [6]:
# reduce dimensionality
train, valid, test = reduce(mnist_train, mnist_valid, mnist_test, 'bin')

### KNN

In [55]:
# test on validation set
valid_accuracy, valid_precision, valid_recall, valid_f1, confusion = evaluate_knn(train, valid, 'euclidean')
print('Validation Set Performance')
print('Accuracy:', valid_accuracy)
print('Precision:', valid_precision)
print('Recall:', valid_recall)
print('F1:', valid_f1)

Validation Set Performance
Accuracy: 0.85
Precision: 0.8586694007746638
Recall: 0.8525780318715102
F1: 0.8500070140431186


In [56]:
# test set performance
test_accuracy, test_precision, test_recall, test_f1, confusion = evaluate_knn(train, test, 'euclidean')
print('Test Set Performance')
print('Accuracy:', test_accuracy)
print('Precision:', test_precision)
print('Recall:', test_recall)
print('F1:', test_f1)

print(confusion)

Test Set Performance
Accuracy: 0.895
Precision: 0.89490409233869
Recall: 0.885273518878782
F1: 0.8849462086495985
[[18  0  0  0  0  0  0  0  0  0]
 [ 0 27  0  0  0  0  0  0  0  0]
 [ 0  1 16  0  0  0  0  1  1  0]
 [ 0  0  0 17  0  0  0  0  1  0]
 [ 0  0  0  0 24  0  0  0  0  1]
 [ 0  0  0  3  0  8  1  0  1  0]
 [ 0  0  0  0  0  0 13  0  0  0]
 [ 0  1  0  0  0  0  0 23  0  0]
 [ 1  1  1  1  0  1  0  0 15  1]
 [ 0  2  0  0  1  0  1  0  0 18]]


### K-Means

In [16]:
labels = [i[0] for i in valid]
data = [i[1] for i in valid]
starter.kmeans_eric(train, data, 'euclidean')


['g',
 'c',
 'g',
 'c',
 'i',
 'e',
 'g',
 'i',
 'c',
 'e',
 'd',
 'i',
 'i',
 'd',
 'i',
 'd',
 'j',
 'd',
 'd',
 'b',
 'h',
 'b',
 'i',
 'f',
 'i',
 'c',
 'g',
 'd',
 'a',
 'c',
 'c',
 'h',
 'h',
 'f',
 'd',
 'c',
 'b',
 'h',
 'c',
 'g',
 'e',
 'd',
 'f',
 'a',
 'a',
 'c',
 'i',
 'f',
 'e',
 'i',
 'c',
 'd',
 'a',
 'b',
 'b',
 'j',
 'c',
 'e',
 'j',
 'b',
 'f',
 'c',
 'g',
 'd',
 'c',
 'c',
 'g',
 'a',
 'e',
 'h',
 'b',
 'd',
 'h',
 'h',
 'e',
 'h',
 'c',
 'c',
 'a',
 'd',
 'a',
 'f',
 'j',
 'h',
 'e',
 'j',
 'c',
 'f',
 'i',
 'i',
 'd',
 'i',
 'j',
 'a',
 'b',
 'b',
 'f',
 'g',
 'f',
 'e',
 'h',
 'j',
 'a',
 'a',
 'i',
 'h',
 'f',
 'h',
 'j',
 'i',
 'j',
 'c',
 'j',
 'c',
 'c',
 'f',
 'c',
 'i',
 'e',
 'a',
 'a',
 'a',
 'j',
 'a',
 'e',
 'i',
 'j',
 'f',
 'h',
 'a',
 'b',
 'g',
 'b',
 'j',
 'a',
 'd',
 'd',
 'd',
 'j',
 'h',
 'g',
 'b',
 'c',
 'b',
 'g',
 'e',
 'c',
 'i',
 'i',
 'a',
 'g',
 'h',
 'c',
 'c',
 'd',
 'f',
 'g',
 'j',
 'j',
 'a',
 'g',
 'i',
 'h',
 'c',
 'd',
 'e',
 'd'